# Imports


In [ ]:
import os
import cv2
import glob
import PIL
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from skimage import data
from skimage.util import montage
import skimage.transform as skTrans
from skimage.transform import rotate
from skimage.transform import resize
from PIL import Image, ImageOps

!pip install nibabel nilearn
import nilearn as nl
import nibabel as nib
import nilearn.plotting as nlplt

import keras
import keras.backend as K
from keras.callbacks import CSVLogger
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers import *
import tensorflow.keras as tfk

np.set_printoptions(precision=3, suppress=True)
K.clear_session()

# Constants


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


TRAIN_DATASET_PATH='/content/drive/My Drive/data/isles/rawdata/'
VALIDATION_DATASET_PATH=''

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
SEGMENT_CLASSES = {
    0 : 'NOT LESION',
    1 : 'LESION'
}

In [ ]:
IMG_SIZE=128
NR_SLICES = 32

In [ ]:
config = dict()

config["data_folder"] = os.path.abspath("/content/drive/My Drive/data/isles/rawdata/") #path to folder containing the training data
config["model_file"] = os.path.join(config["model_folder"], "model.h5")
config["wieghts_file_bestval"] = os.path.join(config["model_folder"], "BEST_val.h5")
config["wieghts_file_lasttrain"] = os.path.join(config["model_folder"], "LAST_train.h5")
config["training_file"] = os.path.join(config["output_folder"], "training_ids.pkl")
config["validation_file"] = os.path.join(config["output_folder"], "validation_ids.pkl")
config["logging_file"] = os.path.join(config["output_folder"], "training.log")
config["overwrite"] = False  # If True, will overwite previous files. If False, will use previously written files.

# model settings
config["input_size"] = (IMG_SIZE,IMG_SIZE) # size of image (x,y)
config["nr_slices"] = (NR_SLICES,) # all inputs will be resliced to this number of slices (z axis), must be power of 2
config["modalities"] = ["dwi"]
config["mean"] = [37.3555097]#preprocessed
config["std"] = [84.6591447]#preprocessed

# Mean:  [37.3555097]
# STD:  [84.6591447]

config["labels"] = 1 # the label numbers on the input image (exclude background)

# training settings
config["batch_size"] = 16
config["n_epochs"] = 200  # cutoff the training after this many epochs
config["initial_learning_rate"] = 5e-3
config["learning_rate_drop"] = 0.5  # factor by which the learning rate will be reduced
config["test_size"] = 0.2  # portion of the data that will be used for validation

# Training Metrics


In [ ]:
def dice_coef(y_true, y_pred, smooth=1.0):
    class_num = 2
    for i in range(class_num):
        y_true_f = keras.layers.Flatten()(y_true[:,:,:,i])
        y_pred_f = keras.layers.Flatten()(y_pred[:,:,:,i])
        intersection = tf.reduce_sum(y_true_f * y_pred_f)
        loss = ((2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth))
        if i == 0:
            total_loss = loss
        else:
            total_loss = total_loss + loss
    total_loss = total_loss / class_num
    return total_loss

# Model

In [ ]:
def conv_block(x, kernels, kernel_size=(3, 3), strides=(1, 1), padding='same',
               is_bn=True, is_relu=True, n=2):
    """ Custom function for conv2d:
        Apply  3*3 convolutions with BN and relu.
    """
    for i in range(1, n + 1):
        x = tfk.layers.Conv2D(filters=kernels, kernel_size=kernel_size,
                            padding=padding, strides=strides,
                            kernel_regularizer=tf.keras.regularizers.l2(1e-4),
                            kernel_initializer=tfk.initializers.he_normal(seed=5))(x)
        if is_bn:
            x = tfk.layers.BatchNormalization()(x)
        if is_relu:
            x = tfk.activations.relu(x)

    return x


In [ ]:
# source https://naomi-fridman.medium.com/multi-class-image-segmentation-a5cc671e647a


def unet3plus(input_shape, output_channels):
    """ UNet3+ base model """
    filters = [8, 16, 32, 64, 128]

    input_layer = tfk.layers.Input(
        shape=input_shape,
        name="input_layer"
    )  # 320*320*3

    """ Encoder"""
    # block 1
    e1 = conv_block(input_layer, filters[0])  # 320*320*64

    # block 2
    e2 = tfk.layers.MaxPool2D(pool_size=(2, 2))(e1)  # 160*160*64
    e2 = conv_block(e2, filters[1])  # 160*160*128

    # block 3
    e3 = tfk.layers.MaxPool2D(pool_size=(2, 2))(e2)  # 80*80*128
    e3 = conv_block(e3, filters[2])  # 80*80*256

    # block 4
    e4 = tfk.layers.MaxPool2D(pool_size=(2, 2))(e3)  # 40*40*256
    e4 = conv_block(e4, filters[3])  # 40*40*512

    # block 5
    # bottleneck layer
    e5 = tfk.layers.MaxPool2D(pool_size=(2, 2))(e4)  # 20*20*512
    e5 = conv_block(e5, filters[4])  # 20*20*1024

    """ Decoder """
    cat_channels = filters[0]
    cat_blocks = len(filters)
    upsample_channels = cat_blocks * cat_channels

    """ d4 """
    e1_d4 = tfk.layers.MaxPool2D(pool_size=(8, 8))(e1)  # 320*320*64  --> 40*40*64
    e1_d4 = conv_block(e1_d4, cat_channels, n=1)  # 320*320*64  --> 40*40*64

    e2_d4 = tfk.layers.MaxPool2D(pool_size=(4, 4))(e2)  # 160*160*128 --> 40*40*128
    e2_d4 = conv_block(e2_d4, cat_channels, n=1)  # 160*160*128 --> 40*40*64

    e3_d4 = tfk.layers.MaxPool2D(pool_size=(2, 2))(e3)  # 80*80*256  --> 40*40*256
    e3_d4 = conv_block(e3_d4, cat_channels, n=1)  # 80*80*256  --> 40*40*64

    e4_d4 = conv_block(e4, cat_channels, n=1)  # 40*40*512  --> 40*40*64

    e5_d4 = tfk.layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(e5)  # 80*80*256  --> 40*40*256
    e5_d4 = conv_block(e5_d4, cat_channels, n=1)  # 20*20*1024  --> 20*20*64

    d4 = tfk.layers.concatenate([e1_d4, e2_d4, e3_d4, e4_d4, e5_d4])
    d4 = conv_block(d4, upsample_channels, n=1)  # 40*40*320  --> 40*40*320

    """ d3 """
    e1_d3 = tfk.layers.MaxPool2D(pool_size=(4, 4))(e1)  # 320*320*64 --> 80*80*64
    e1_d3 = conv_block(e1_d3, cat_channels, n=1)  # 80*80*64 --> 80*80*64

    e2_d3 = tfk.layers.MaxPool2D(pool_size=(2, 2))(e2)  # 160*160*256 --> 80*80*256
    e2_d3 = conv_block(e2_d3, cat_channels, n=1)  # 80*80*256 --> 80*80*64

    e3_d3 = conv_block(e3, cat_channels, n=1)  # 80*80*512 --> 80*80*64

    e4_d3 = tfk.layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(d4)  # 40*40*320 --> 80*80*320
    e4_d3 = conv_block(e4_d3, cat_channels, n=1)  # 80*80*320 --> 80*80*64

    e5_d3 = tfk.layers.UpSampling2D(size=(4, 4), interpolation='bilinear')(e5)  # 20*20*320 --> 80*80*320
    e5_d3 = conv_block(e5_d3, cat_channels, n=1)  # 80*80*320 --> 80*80*64

    d3 = tfk.layers.concatenate([e1_d3, e2_d3, e3_d3, e4_d3, e5_d3])
    d3 = conv_block(d3, upsample_channels, n=1)  # 80*80*320 --> 80*80*320

    """ d2 """
    e1_d2 = tfk.layers.MaxPool2D(pool_size=(2, 2))(e1)  # 320*320*64 --> 160*160*64
    e1_d2 = conv_block(e1_d2, cat_channels, n=1)  # 160*160*64 --> 160*160*64

    e2_d2 = conv_block(e2, cat_channels, n=1)  # 160*160*256 --> 160*160*64

    d3_d2 = tfk.layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(d3)  # 80*80*320 --> 160*160*320
    d3_d2 = conv_block(d3_d2, cat_channels, n=1)  # 160*160*320 --> 160*160*64

    d4_d2 = tfk.layers.UpSampling2D(size=(4, 4), interpolation='bilinear')(d4)  # 40*40*320 --> 160*160*320
    d4_d2 = conv_block(d4_d2, cat_channels, n=1)  # 160*160*320 --> 160*160*64

    e5_d2 = tfk.layers.UpSampling2D(size=(8, 8), interpolation='bilinear')(e5)  # 20*20*320 --> 160*160*320
    e5_d2 = conv_block(e5_d2, cat_channels, n=1)  # 160*160*320 --> 160*160*64

    d2 = tfk.layers.concatenate([e1_d2, e2_d2, d3_d2, d4_d2, e5_d2])
    d2 = conv_block(d2, upsample_channels, n=1)  # 160*160*320 --> 160*160*320

    """ d1 """
    e1_d1 = conv_block(e1, cat_channels, n=1)  # 320*320*64 --> 320*320*64

    d2_d1 = tfk.layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(d2)  # 160*160*320 --> 320*320*320
    d2_d1 = conv_block(d2_d1, cat_channels, n=1)  # 160*160*320 --> 160*160*64

    d3_d1 = tfk.layers.UpSampling2D(size=(4, 4), interpolation='bilinear')(d3)  # 80*80*320 --> 320*320*320
    d3_d1 = conv_block(d3_d1, cat_channels, n=1)  # 320*320*320 --> 320*320*64

    d4_d1 = tfk.layers.UpSampling2D(size=(8, 8), interpolation='bilinear')(d4)  # 40*40*320 --> 320*320*320
    d4_d1 = conv_block(d4_d1, cat_channels, n=1)  # 320*320*320 --> 320*320*64

    e5_d1 = tfk.layers.UpSampling2D(size=(16, 16), interpolation='bilinear')(e5)  # 20*20*320 --> 320*320*320
    e5_d1 = conv_block(e5_d1, cat_channels, n=1)  # 320*320*320 --> 320*320*64

    d1 = tfk.layers.concatenate([e1_d1, d2_d1, d3_d1, d4_d1, e5_d1, ])
    d1 = conv_block(d1, upsample_channels, n=1)  # 320*320*320 --> 320*320*320

    # last layer does not have batchnorm and relu
    d = conv_block(d1, output_channels, n=1, is_bn=False, is_relu=False)

    output = tfk.activations.softmax(d)

    return tf.keras.Model(inputs=input_layer, outputs=[output], name='UNet_3Plus')


model = unet3plus((IMG_SIZE, IMG_SIZE, 2), 2)

# Data Loader

In [ ]:
class NiiDatasetLoader:
    def __init__(self, preprocessors=None):
        # store the image preprocessor
        self.preprocessors = preprocessors

        # if the preprocessors are None, initialize them as an
        # empty list
        if self.preprocessors is None:
            self.preprocessors = []

    def load(self, data_folder, modalities):
        # initialize the list of features and labels
        data = []
        labels = []
        for folder in sorted(glob.glob(os.path.join(data_folder, "*"))):
            # show an update info
            print('[INFO] Processing foder: ' + folder)
            img = []
            for modality in modalities:
                filename = glob.glob(os.path.join(folder, '*%s*.nii' % modality))[0]
                image = nib.load(filename).get_fdata()
                # check to see if our preprocessors are not None
                if self.preprocessors is not None:
                    # loop over the preprocessors and apply each to the image
                    for p in self.preprocessors:
                        image = p.preprocess(image)
                img.append(image)
            data.append(np.array(img))
            filename = glob.glob(os.path.join(folder, '*seg*.nii'))
            if filename == []:
                label = None
            else:
                label= nib.load(filename[0]).get_fdata()
                # check to see if our preprocessors are not None
                if self.preprocessors is not None:
                    # loop over the preprocessors and apply each to the image
                    for p in self.preprocessors:
                        label = p.preprocess(label)
                label = np.expand_dims(label, axis=0)
            labels.append(np.array(label))
        return (data, labels)

In [ ]:
(data, labels) = NiiDatasetLoader(preprocessor).load(config["data_folder"], config["modalities"])

# Training

In [ ]:
csv_logger = CSVLogger('training_20.log', separator=',', append=False)


callbacks = [
      keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1),

        csv_logger
    ]

In [ ]:
model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics = [dice_coef])
# Evaluate the model on the test data using `evaluate`

In [ ]:
K.clear_session()
model.fit(
    training_generator,
    epochs=200,
    steps_per_epoch=len(train_ids),
    batch_size=8,
    validation_data=valid_generator,
    callbacks=callbacks
)

Epoch 1/200


In [ ]:
model.save("200epochsUNET3+112.keras")